In [2]:
import requests
import pandas as pd

r = requests.get('https://gutenberg.org/files/786/786-0.txt')
r.text
r.encoding = 'utf-8'
print(r.text)



The Project Gutenberg eBook, Hard Times, by Charles Dickens


This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org





Title: Hard Times


Author: Charles Dickens



Release Date: March 17, 2013  [eBook #786]
[This file was first posted on January 20, 1997]

Language: English

Character set encoding: UTF-8


***START OF THE PROJECT GUTENBERG EBOOK HARD TIMES***


Transcribed from the 1905 Chapman and Hall edition by David Price, email
ccx074@pglaf.org





                                HARD TIMES
                                   AND
                           REPRINTED PIECES {0}


                                * * * * *

                            By CHARLES DICKENS

                                * * * * *

             _With illustrations by Marcus Stone_, _Maurice_
               

In [9]:
print(dickens_books)

[46, 98, 564, 580, 588, 644, 650, 653, 675, 676, 678, 699, 700, 730, 766, 786, 807, 809, 810, 821, 824, 872, 882, 883, 888, 912, 914, 916, 917, 918, 922, 924, 927, 963, 967, 968, 1023, 1289, 1392, 1394, 1400, 1406, 1407, 1413, 1414, 1415, 1416, 1419, 1421, 1422, 1423, 1435, 1464, 1465, 1467, 1831, 2303, 2324, 7869, 8608, 9689, 9690, 9691, 9692, 9693, 9694, 9695, 9696, 9697, 9698, 9699, 9702, 9703, 9704, 9705, 9706, 9707, 9708, 9709, 9710, 9711, 9712, 9713, 9714, 9715, 9716, 9717, 9718, 9719, 9720, 9721, 9722, 9723, 9724, 9725, 9726, 9727, 9728, 9729, 9730, 9731, 9732, 9733, 9734, 9735, 9736, 9737, 9738, 9739, 9740, 9741, 9742, 9743, 9744, 10135, 13771, 14789, 15499, 15618, 16020, 16021, 16022, 16023, 17565, 17675, 17676, 17868, 17869, 17879, 17880, 18059, 18186, 18201, 19337, 19505, 19934, 20673, 20795, 22449, 22465, 23342, 23344, 23452, 23765, 24022, 24026, 24501, 25852, 25853, 25854, 25985, 27924, 28198, 28560, 30127, 30368, 31517, 31638, 32241, 32372, 35536, 37093, 37121, 37581, 407

In [7]:
Catalog = pd.read_csv("pg_catalog.csv")
#print(Catalog.head(5))
column_name = "Authors"
list_of_authors = Catalog[column_name]
print(list_of_authors)
dickens_books=[]
book_indexs = Catalog["Text#"]
for index, author in enumerate(list_of_authors):
    if type(author)==float:
        continue        
    if "Dickens, Charles" in author:
        dickens_books.append(book_indexs[index])
#for index in dickens_books:
for books in dickens_books: 
    r = requests.get(f'https://gutenberg.org/files/{books}/{books}-0.txt')
    r.encoding = 'utf-8'
    with open(f'BooksRaw/Book{books}.txt', 'w') as f:
        f.write(r.text)

/home/jprism/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


0                             Jefferson, Thomas, 1743-1826
1                                            United States
2            Kennedy, John F. (John Fitzgerald), 1917-1963
3                              Lincoln, Abraham, 1809-1865
4                                            United States
                               ...                        
67491                                    Whitman, Roger B.
67492    Fern, Fanny, 1811-1872; Coffin, Fred M. [Illus...
67493       O'Connor, Patrick; Campbell, Ray [Illustrator]
67494                                    Parmanen, Eino I.
67495                                                  NaN
Name: Authors, Length: 67496, dtype: object
